In [ ]:
!pip install langchain==0.1.16
!pip install faiss-cpu
!pip install sentence-transformers
!pip install pypdf
!pip install transformers accelerate torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 85.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.1/303.1 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 85.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.1.2
    Uninstalling tenacity-9.1.2:
      Successfully uninstalled tenacity-9.1.2
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
  Attempting uninstall: numpy
    Found exist

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.0/329.0 kB 27.9 MB/s eta 0:00:00


In [ ]:
from google.colab import files

uploaded = files.upload()

Saving A Comprehensive Review on Fake News Detection With Deep Learning.pdf to A Comprehensive Review on Fake News Detection With Deep Learning.pdf
Saving applsci-10-00370-v2.pdf to applsci-10-00370-v2.pdf
Saving electronics-12-04261.pdf to electronics-12-04261.pdf


In [ ]:
import os
from langchain.document_loaders import PyPDFLoader

documents = []

for file in os.listdir():
    if file.endswith(".pdf"):
        loader = PyPDFLoader(file)
        pages = loader.load()
        documents.extend(pages)
        print(f"Loaded {file} with {len(pages)} pages")

print("Total pages loaded:", len(documents))

Loaded electronics-12-04261.pdf with 26 pages
Loaded A Comprehensive Review on Fake News Detection With Deep Learning.pdf with 20 pages
Loaded applsci-10-00370-v2.pdf with 14 pages
Total pages loaded: 60


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=200
)

chunks = text_splitter.split_documents(documents)

print("Total chunks created:", len(chunks))

Total chunks created: 440


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

vectorstore = FAISS.from_documents(chunks, embeddings)

print("FAISS vector store created successfully")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

FAISS vector store created successfully


In [ ]:
from transformers import pipeline

llm = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    max_length=512
)

print("LLM loaded successfully")

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


LLM loaded successfully


In [ ]:
def rag_answer(question, vectorstore, k=4):
    # 1. Retrieve relevant chunks
    docs = vectorstore.similarity_search(question, k=k)

    # 2. Combine retrieved text
    context = "\n\n".join([doc.page_content for doc in docs])

    # 3. Prompt
    prompt = f"""
    You are an academic research assistant.

    Use ONLY the context below to answer the question.
    If the answer is not present, say so clearly.

    Context:
    {context}

    Question:
    {question}

    Answer:
    """

    # 4. Generate answer
    response = llm(prompt)

    return response[0]["generated_text"]

In [ ]:
question = "What neural network architectures are commonly used in these papers, and why are they suitable for text data?"

answer = rag_answer(question, vectorstore)

print("QUESTION:")
print(question)

print("\nANSWER:")
print(answer)

QUESTION:
What neural network architectures are commonly used in these papers, and why are they suitable for text data?

ANSWER:
CNN in section V -Aand RNN in Section V -B, Graph Neural Network in SectionV -C, Generative Adversarial Network in Section V -D, Attention sequentially connected to construct a directed graph. The output from the earlier step serves as the input to the cur- rent step. RNNs are effective in time and sequence-based predictions. RNN is less compatible with features compared to CNN. RNNs are suitable for studying sequential texts and expressions. However, it cannot process very long sequences when tanh or ReLU is used as an activation function. The backward-propagation algorithm is utilized in the RNN for training. While training the neural networks, it is required to take tiny steps frequently in the way of the nega- tive error derivative concerning network weights to establish FIGURE 7. The figure shows an architecture of basic RNN with n sequential layers. x r